# Muscle-bids: dcm2mbids

By [ENTER DATA](ENTER DATA), 2022  
Notebook license: ENTER DATA 
Code license: ENTER DATA

---

## Package description

The [muscle-bids package](https://github.com/fsantini/muscle-bids/tree/Jupyter) converts traditional MRI data in DICOM format into a [BIDS](https://bids.neuroimaging.io/)-compatible structure. It can be ran as a standalone executable or called within Python scripts as a library.


## Dependencies and installation

Before using this package, make sure that the packages below are installed in your environment:   
        - [Python 3](https://www.python.org/downloads/)  
        - [Bidict](https://pypi.org/project/bidict/)
        - [H5py](https://www.h5py.org/)  
        - [Natsort](https://pypi.org/project/natsort/)  
        - [NiBabel](https://nipy.org/nibabel/)  
        - [NumPy](https://numpy.org/)  
        - [SciPy](https://scipy.org/)  
        - [Pandas](https://pandas.pydata.org/)  
        - [Pydicom](https://pydicom.github.io/)  
        - [Pyparsing](https://github.com/pyparsing/pyparsing)  
        - [Python-dateutil](https://pypi.org/project/python-dateutil/)  
        - [Pytz](https://pypi.org/project/pytz/)  
        - [Six](https://pypi.org/project/six/)  
        - [Tqdm](https://tqdm.github.io/)  
        - [SimpleITK](https://simpleitk.org/)  
        - [CuPy](https://cupy.dev/)  
          
To install the muscle-bids package, run the code below:

In [ ]:
pip install dist/muscle-bids-0.0.1.tar.gz

You are now ready to start using the muscle-bids package to standardise your data according to BIDS!

## Run as executable

The Python package is set up in such a way that after installing you can use the code in your own code, and directly from the command line as well.

The commandline script is called 'dcm2mbids.exe' and is by default stored in the same folder of your python executable (and should be added to your system's PATH variable. 

> *Syntax: dcm2mbids link_to_dicom_folder link_to_output_folder*

If no output folder exists, the program will create it.

The folder with example files contains a Spin Echo MRI image. Open a command window with in the current directory and run following command:



  ## Use as python module





The muscle-bids package can also be used as an imported module within your pythons scripts. The example below explains how to process your data in this way.

First, import all the necessary packages:

In [ ]:
import os
import sys

from muscle_bids.dosma_io import MedicalVolume, DicomReader
from muscle_bids.utils.headers import reduce
from muscle_bids.utils.io import load_dicom, save_bids, load_bids, save_dicom

import muscle_bids.converters

Then, you will need to specify the directories for your input and output data:

In [ ]:
INPUT_FOLDER = '.\MESE_Anon'
OUTPUT = '.\MESE_Nii\test.nii.gz'

You can now load your DICOM images and save them as bids as in the example below:

In [ ]:
med_volume = load_dicom(INPUT_FOLDER, 'EchoTime')
print(med_volume.shape)
save_bids(OUTPUT, med_volume)